In [ ]:
#Importamos las librerías necesarias
!git clone https://github.com/JaimeRosado/LegPy.git
import sys
#sys.path.insert(0,'/content/LegPy')
sys.path.insert(0,'LegPy')
import os
import numpy as np
import pandas as pd
import LegPy as lpy
Database = 'IAEA' # Lista de bases de datos disponibles: 'MIRD' o 'IAEA'
Isotopo = '111In' # Lista de isotopos disponibles:  '90Y', '99mTc', '111In', '161Tb', '177Lu' o '181Re'
Isotopos_con_espectro_continuo = ('188Re', '186Re' , '177Lu', '161Tb', '99mTc', '90Y') # Isotopos de la lista de isotopos disponible que emiten electrones beta
Isotopos_con_espectro_discreto = ('188Re', '186Re' , '177Lu', '161Tb', '111In', '99mTc', '90Y') # Isotopos de la lista de isotopos disponible que emiten electrones Auger o conversión interna
medium = lpy.Medium(name='Water, Liquid') # Definimos en LegPy la composición química del medio sobre el que se va a hacer la simulación. En este caso partícular es agua
E = 0.1 # Energía, en MeV, en la que se va a a calcular el rango CSDA usado de los electrones emitidos en la simulación
e_data = medium.e_data # Importamos el rango CSDA del medio
CSDA = np.interp(E, e_data.E_ref, e_data.R_ref) # Interpolamos el valor del CSDA para la energía en cuestión
CSDAum = CSDA * 1e4 # Pasamos el CSDA a micrómetros
diameter_list = [10000, 5000, 2000, 1000, 500, 200, 100, 50, 20, 10] # Lista con los diámetros de las esferas usadas en la referencia
e_length = [CSDAum * 0.01] *len(diameter_list) # Paso de los electrones en cada interacción. Se toma por comodidad una centésima parte del rango CSDA
# Función donde cargamos los espectros de emisión de electrones, ya sea los beta o los Auger y conversión interna, procedente de un archivo .txt con los datos. Sirve para ambos
def cargar_txt(nombre_archivo):
    if not os.path.exists(nombre_archivo):
        raise FileNotFoundError(f'Archivo no encontrado: {nombre_archivo}')
    return np.loadtxt(nombre_archivo)

# Función para definir en LegPy los espectros de emisión de electrones  a partir de los cargados usando la función anterior
def obtener_spectros(isotopo):
    espectros = {}
    if isotopo in Isotopos_con_espectro_discreto:
        E_w = cargar_txt(Isotopo + '_' + Database + '_CEAU_corregido.txt') # Aquí se define en LegPy el espectro beta, si lo tuviera
        espectros["discreto"] = lpy.Spectrum(name='multi_mono', E_w=E_w)
    if isotopo in Isotopos_con_espectro_continuo:
        espectros["continuo"] = lpy.Spectrum(name='from_file', file= Isotopo + '_' + Database + '_Beta') # Aquí se define en LegPy el espectro Auger y conversión interna, si lo tuviera
    return espectros
# Función para realizar la simulación a partir de las mismas condiciones que las de la referencia. La función pide el diámetro de la esfera (en micrómetros), la composición química del medio, el
# número de electrones emitidos por desintegración, el paso de los electrones, y el número de partículas a simular
def simular_dosis(d, medium, espectros, normalizacion, e_length, n_part=200000):
    R_um = d / 2 # Radio de la esfera, en micrómetros
    R = R_um / 1e4 # Radio de la esfera, en centímetros
    V_um3 = (4/3) * np.pi * R_um**3 # Volumen de la esfera en micrómetros cúbicos
    x_r = 0.9 * R_um / e_length
    n_r = int(x_r) # Número de bines radiales de la esfera
    if n_r == 0:
        n_r = 1
    geometry = lpy.Geometry(name='sphere', r=R, n_r=n_r) # Definimos en LegPy la geometría del medio. En este caso partícular es una esfera
    beam = lpy.Beam(name='isotropic', particle='electron', p_in=[0., 0., 0.], r_s=R) # Definimos en LegPy la geometría del haz de partículas. En este caso se trata de una distribución
    # uniforme a lo largo de la esfera de fuentes puntuales
    E_mean = 0 # Variable para almacenar la energía depositada por volumen, en keV/cm^3, en cada bin de la esfera
    if 'continuo' in espectros:
        Edep_cont = lpy.MC(medium, geometry, espectros['continuo'], beam, n_part, e_length, e_transport = True).Edep # Se ejecuta el monte Carlo para los electrones Auger y conversión interna, si los hubiera
        E_mean += normalizacion[1] * Edep_cont # Sumamos a la energía depositada por volumen en cada bin de la esfera, en keV/cm^3, la depositada por los electrones Auger y converión interna
    if 'discreto' in espectros:
        Edep_dis = lpy.MC(medium, geometry, espectros['discreto'], beam, n_part, e_length, e_transport = True).Edep # Se ejecuta el monte Carlo para los electrones beta, si los hubiera
        E_mean += normalizacion[2] * Edep_dis   # Sumamos a la energía depositada por volumen en cada bin de la esfera, en keV/cm^3, la depositada por los electrones beta
    E_mean /= normalizacion[0]
    Vp = geometry.delta_v # Lista con el volumen de cada bin de la esfera, en cm^3
    E = Vp*E_mean # Multiplicamos la energía depositada por volumen en cada bin por el volumen de cada bin para obtener la energía depositada en cada bin de la esfera, en keV
    E_dep = E.sum() # Sumamos todas las contribuciones para obtener la energía depositada en la esfera, en keV
    dose_e = normalizacion[0] * E_dep * 1.602e-16 / (1e-3 * (4/3) * np.pi * R**3 ) # Dividimos la energía depositada, en J, entre la masa de la esfera, en kg, para obtener la dosis depositada, en Gy
    dose_um3 = dose_e * V_um3 # Multiplicamos por el volumen para obtener la dosis depositada por los electrones para una desintegración por micrómetro cúbico
    print('Diámetro de la esfera = ', d, ' micrómetros')
    print('Númer de bines radiales = ', n_r)
    print('Dosis depositada por los electrones para una desintegración por micrómetro cúbico: ', dose_um3, ' Gy')
    print('---')
    return [2 * R_um, dose_um3 * normalizacion[0]] # Devolvemos una lista con dos valores: El diámetro de la esfera y la dosis depositada por los electrones para una desintegración por micrómetro cúbico
normalizacion = cargar_txt(Isotopo + '_' + Database + '_normalizacion.txt')
espectros = obtener_spectros(Isotopo)
print('Se va a calcular la dosis depositada por los electrones para una distribución uniforme de 1 desintegración por micrómetro cúbico del isótopo ',Isotopo, ' a partir de la base de datos ', Database)
print('Datos de la simuación:')
print(f'CSDA usado: ', CSDA, ' cm')
print('Número de electrones emitidos por desintegración: ', normalizacion[0])
print('Número de electrones beta emitidos por desintegración:', normalizacion[1])
print('Número de electrones Auger o conversión interna emitidos por desintegración: ', normalizacion[2])
print('---')
resultados = [simular_dosis(d, medium, espectros, normalizacion, e_length) for d, e_length in zip(diameter_list, e_length)] # Se llama a la función para realizar la simulación

fatal: destination path 'LegPy' already exists and is not an empty directory.
Se va a calcular la dosis depositada por los electrones para una distribución uniforme de 1 desintegración por micrómetro cúbico del isótopo  111In  a partir de la base de datos  IAEA
Datos de la simuación:
CSDA usado:  0.01431  cm
Número de electrones emitidos por desintegración:  1.462821
Número de electrones beta emitidos por desintegración: 0.0
Número de electrones Auger o conversión interna emitidos por desintegración:  1.462821
---
The step length of some electrons may be greater than the voxel size.
